In [1]:
import os

os.environ['KERAS_BACKEND'] = "torch"

In [2]:
import dataset

lazy = dataset.LazyCommentDataset("../data/corpus.preprocessed.csv")
dt = dataset.CommentDataset("../data/corpus.preprocessed.csv")

Index        15522432
comments    379211268
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 1940304 entries, 0 to 1940425
Data columns (total 1 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   comments  object
dtypes: object(1)
memory usage: 376.4 MB
None


Without preprocessing on dataset: 379211268 <br />
If the dataset is processed already: 677671688 (Twice the size) <br />

In [3]:
from torch.utils.data import DataLoader

dt_dataloader = DataLoader(dt, batch_size=32, shuffle=True, collate_fn=lambda x: x)
lazy_dataloader = DataLoader(lazy, batch_size=32, shuffle=True, collate_fn=lambda x: x)

In [4]:
dt_iterator = iter(dt_dataloader)
lazy_iterator = iter(lazy_dataloader)

In [5]:
next(dt_iterator)

[['love',
  'play',
  'game',
  'kiddo',
  '.',
  '  ',
  'fast',
  '.',
  '  ',
  'bit',
  'random',
  '/',
  'fast',
  'k',
  ',',
  'work',
  'pretty',
  '.'],
 ['deep',
  'stunningly',
  'beautiful',
  '.',
  'good',
  'mix',
  'economic',
  'betting',
  '.',
  'annoyingly',
  'short',
  'sense',
  'achieve',
  'goal',
  '-',
  '6',
  'turn',
  'develop',
  'proper',
  'strategy',
  '.',
  'hear',
  'expansion',
  'enable',
  '8',
  '-',
  'turn',
  'game',
  ',',
  'slightly',
  'well',
  '.'],
 ['challenging',
  'game',
  ',',
  'great',
  'couple',
  'gamer',
  '...',
  'gamer',
  'couple',
  '.'],
 ['black', 'box', 'version'],
 ['play',
  'time',
  'long',
  'time',
  'friend',
  '.',
  '  ',
  'manage',
  'tie',
  'second',
  'place',
  ',',
  'thank',
  'strategy',
  'farm',
  'lot',
  'unloved',
  'bean',
  ',',
  'buy',
  'field',
  'quickly',
  '.',
  '  ',
  'fun',
  'game',
  ',',
  'work',
  'large',
  'group',
  '.',
  '  '],
 ['way',
  'long',
  '.',
  'quick',
  'exp

In [7]:
next(lazy_iterator)

[['instant',
  'classic',
  '.',
  '  ',
  'feel',
  'like',
  'ticket',
  'ride',
  'carcassonne',
  'simplicity',
  'player',
  'interaction',
  'well',
  'replayability',
  '.',
  '  ',
  'favorite',
  'race',
  'game',
  '.'],
 ['1',
  'play',
  '.',
  'enjoy',
  '.',
  'obviously',
  'game',
  'require',
  'play',
  'start',
  'hang',
  '.',
  'rating',
  '.',
  'guess',
  '.'],
 ['2018', '-', 'purchase', 'gauntlet', 'game'],
 ['receive', 'excited', '.'],
 ['hesitant', 'rate', 'higher', 'play', ',', "'", 'damn', '!', "'"],
 ['probably', 'improvement', '.', 'app', 'addictive', '.'],
 ['hurt',
  '.',
  '  ',
  'game',
  'look',
  'interesting',
  ',',
  'get',
  'way',
  '.',
  '  ',
  ',',
  'mistake',
  'combine',
  'set',
  ',',
  'rough',
  'experience',
  'set',
  ',',
  'store',
  ',',
  'sort',
  'thing',
  '.',
  '  ',
  'sort',
  ',',
  'game',
  'come',
  'lack',
  '.',
  '  ',
  'example',
  ',',
  'hate',
  'combat',
  '-',
  'system',
  'reward',
  'player',
  'control'